1. Load data and model

In [2]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available. Check your setup.")


GPU is available!
Using GPU: NVIDIA GeForce RTX 4080 Laptop GPU


In [3]:
# Import required libraries
import os
from datasets import Dataset

# Function to read IOB files and split into paragraphs
def read_iob_file(file_path):
    """Read IOB file and convert to token-label samples"""
    examples = []
    words, labels = [], []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                token, label = line.split("\t")
                words.append(token)
                labels.append(label.strip())
            else:
                # End of a sample
                if words and labels:
                    examples.append({"tokens": words, "labels": labels})
                    words, labels = [], []
    # Add the last example if file doesn't end with a blank line
    if words and labels:
        examples.append({"tokens": words, "labels": labels})
    return examples



In [4]:
# Load datasets using the updated function
train_path = 'C:\\S24-25\\TxM\\dataset\\train.tsv'
val_path = 'C:\\S24-25\\TxM\\dataset\\val_gold.tsv'
test_path = 'C:\\S24-25\\TxM\\dataset\\test_gold.tsv'
train_data = read_iob_file(train_path)
val_data = read_iob_file(val_path)
test_data = read_iob_file(test_path)

print(f"Train samples: {len(train_data)}, Val samples: {len(val_data)}, Test samples: {len(test_data)}")


Train samples: 998, Val samples: 124, Test samples: 126


Tokenize and Align Labels

In [5]:
from transformers import AutoTokenizer
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# Define label mappings
label_list = ["O", "B-ADR", "I-ADR", "B-DRU", "I-DRU", "B-DIS", "I-DIS", "B-SYM", "I-SYM"]
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, padding=True
    )
    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)  # Ignore padding
            elif word_idx != previous_word_idx:
                aligned_labels.append(label_to_id[label[word_idx]])
            else:
                aligned_labels.append(label_to_id[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(aligned_labels)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Tokenize datasets
train_dataset = Dataset.from_list(train_data).map(tokenize_and_align_labels, batched=True)
test_dataset = Dataset.from_list(test_data).map(tokenize_and_align_labels, batched=True)
val_dataset = Dataset.from_list(val_data).map(tokenize_and_align_labels, batched=True)


Map: 100%|██████████| 124/124 [00:00<00:00, 2724.52 examples/s]


In [6]:
from transformers import AutoModelForTokenClassification


# Load the pre-trained BERT model

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2. Train and evaluate with default parameters on the test set

In [7]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=10,
)


In [8]:
from transformers import Trainer
from evaluate import load
import torch

# Load metric for evaluation
metric = load("seqeval")

def compute_metrics(predictions):
    predictions, labels = predictions
    predictions = torch.argmax(torch.tensor(predictions), dim=2)
    
    # Convert predictions and labels to CPU and then to plain Python integers
    predictions = predictions.cpu().numpy()
    
    true_labels = [
        [id_to_label[label] for label in label_seq if label != -100] 
        for label_seq in labels
    ]
    true_predictions = [
        [id_to_label[int(p)] for (p, l) in zip(prediction, label_seq) if l != -100]  # Convert tensor to int
        for prediction, label_seq in zip(predictions, labels)
    ]
    return metric.compute(predictions=true_predictions, references=true_labels)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()



  5%|▌         | 10/189 [00:04<01:09,  2.59it/s]

{'loss': 1.4865, 'grad_norm': 1.8258541822433472, 'learning_rate': 1.8941798941798943e-05, 'epoch': 0.16}


 11%|█         | 20/189 [00:08<01:03,  2.67it/s]

{'loss': 0.5903, 'grad_norm': 1.1083478927612305, 'learning_rate': 1.7883597883597884e-05, 'epoch': 0.32}


 16%|█▌        | 30/189 [00:12<00:59,  2.68it/s]

...

100%|██████████| 189/189 [01:16<00:00,  3.21it/s]c:\Users\HikaLipp\miniconda3\envs\TxM\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                 
100%|██████████| 189/189 [01:18<00:00,  3.21it/s]

{'eval_loss': 0.2564546465873718, 'eval_ADR': {'precision': 0.41148325358851673, 'recall': 0.5172932330827068, 'f1': 0.45836109260493, 'number': 665}, 'eval_DIS': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 58}, 'eval_DRU': {'precision': 0.8484107579462102, 'recall': 0.8675, 'f1': 0.857849196538937, 'number': 400}, 'eval_SYM': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 50}, 'eval_overall_precision': 0.5550200803212851, 'eval_overall_recall': 0.5890878090366581, 'eval_overall_f1': 0.5715467328370554, 'eval_overall_accuracy': 0.9190595363540569, 'eval_runtime': 1.1124, 'eval_samples_per_second': 113.267, 'eval_steps_per_second': 7.192, 'epoch': 3.0}


100%|██████████| 189/189 [01:19<00:00,  2.37it/s]

{'train_runtime': 79.7466, 'train_samples_per_second': 37.544, 'train_steps_per_second': 2.37, 'train_loss': 0.3579519532975696, 'epoch': 3.0}


TrainOutput(global_step=189, training_loss=0.3579519532975696, metrics={'train_runtime': 79.7466, 'train_samples_per_second': 37.544, 'train_steps_per_second': 2.37, 'total_flos': 782372000176128.0, 'train_loss': 0.3579519532975696, 'epoch': 3.0})

In [9]:
# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)


 88%|████████▊ | 7/8 [00:00<00:00,  8.95it/s]c:\Users\HikaLipp\miniconda3\envs\TxM\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 8/8 [00:01<00:00,  7.98it/s]

Evaluation Results: {'eval_loss': 0.2564546465873718, 'eval_ADR': {'precision': 0.41148325358851673, 'recall': 0.5172932330827068, 'f1': 0.45836109260493, 'number': 665}, 'eval_DIS': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 58}, 'eval_DRU': {'precision': 0.8484107579462102, 'recall': 0.8675, 'f1': 0.857849196538937, 'number': 400}, 'eval_SYM': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 50}, 'eval_overall_precision': 0.5550200803212851, 'eval_overall_recall': 0.5890878090366581, 'eval_overall_f1': 0.5715467328370554, 'eval_overall_accuracy': 0.9190595363540569, 'eval_runtime': 1.131, 'eval_samples_per_second': 111.407, 'eval_steps_per_second': 7.073, 'epoch': 3.0}


3. Train and test on val set

4. HPO